In [22]:
import nltk
import pandas as pd
from nltk import sent_tokenize,word_tokenize

In [42]:
#Get lists of positive and negative words
def get_words(url):
    import requests
    words = requests.get(url).content.decode('latin-1')
    word_list = words.split('\n')
    index = 0
    while index < len(word_list):
        word = word_list[index]
        if ';' in word or not word:
            word_list.pop(index)
        else:
            index+=1
    return word_list

p_url = 'http://ptrckprry.com/course/ssd/data/positive-words.txt'
n_url = 'http://ptrckprry.com/course/ssd/data/negative-words.txt'
positive_words = get_words(p_url)
negative_words = get_words(n_url)

def get_pos_neg_ratio(sent, positive_words, negative_words):
    cpos=0
    cneg=0
    for word in sent:
        if len(word) > 1:
            if not word[1] in ['JJ','JJR','JJS']:
                continue
            if word[0] in positive_words:
                cpos += 1
            if word[0] in negative_words:
                cneg += 1
    total_words = len(sent)
    ppct = cpos/total_words *100
    npct = cneg/total_words * 100
    if cneg == 0:
        p_n_t = 0
    else:
        p_n_t = cpos/cneg
    return ppct,npct,p_n_t

def get_entity_names(text):
    sentences = nltk.sent_tokenize(text)

    def extract_entity_names(t):
        entity_names = []

        if hasattr(t, 'label') and t.label:
            if t.label() == 'NE':
                entity_names.append(' '.join([child[0] for child in t]))
            else:
                for child in t:
                    entity_names.extend(extract_entity_names(child))

        return entity_names
            

    entity_sent_sentiment = pd.DataFrame(columns=['sentence','entity names', 'postive ratio', 'negative ratio', 'pos/neg tatio'])
    for sentence in sentences:
        tokenized_sentence = nltk.word_tokenize(sentence)
        tagged_sentence = nltk.pos_tag(tokenized_sentence)
        chunked_sentence = nltk.ne_chunk_sents([tagged_sentence], binary=True)
        for tree in chunked_sentence:
            print(tree)
            name_entity = extract_entity_names(tree)
            if len(name_entity) > 0:
                ppct,npct,p_n_t = get_pos_neg_ratio(tree, positive_words, negative_words)
                entity_sent_sentiment = entity_sent_sentiment.append({'sentence':sentence,'entity names':set(name_entity), 'postive ratio': ppct, 'negative ratio': npct, 'pos/neg tatio': p_n_t},ignore_index=True)   

    return entity_sent_sentiment

In [ ]:
import os
x = os.listdir('./news')
for files in x:
    text_file = open('./news/'+files, "r").read()
    text_pd = get_entity_names(text_file)
    text_pd.to_csv('./sent/'+files[:-3]+'csv')